## Document classification notebook

This notebook illustrates the lecture on document classification. We will go through the steps towards basic
document classification systems with a bag of words representation and tf-idf weighting associaed with k-nn search.


In [16]:
#
# load a bunch of modules
#

import json
import numpy as np
import spacy
from tqdm import tqdm
process = spacy.load('en_core_web_md')
print(process.pipeline)


[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x2ad83cfa0>), ('tagger', <spacy.pipeline.tagger.Tagger object at 0x2ad83c460>), ('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x2acedc120>), ('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x2acee7240>), ('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer object at 0x2acc7a540>), ('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x2acedca50>)]


### Data

The data that we will use comes from a super popular toy dataset for opinion mining: IMDb. The dataset provides reviews from the IMDb website where half of the reviews express positive comments and half negative. The task is this geared towards detecting the _polarity_ of the opinion expressed.

See https://ai.stanford.edu/~amaas/data/sentiment for details. You can also get the raw data from there but I'm providing you with an easier json version to load

In [2]:
#
# load IMDb data
#
fn = '../TP/data/imdb-trn.json'

with open(fn, 'rt') as f:
    imdb_data = json.load(f)
    
print('number of utterances =', len(imdb_data))
print('number of + utterances =', len([x for x in imdb_data if x[0] == 'pos']))
print('number of - utterances =', len([x for x in imdb_data if x[0] == 'neg']))
print('data[0] =', imdb_data[0])
print('data[-1] =', imdb_data[-1])


number of utterances = 25000
number of + utterances = 12500
number of - utterances = 12500
data[0] = ['pos', 'For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.']
data[-1] = ['neg', 'Not that I dislike childrens movies, but this was a tearjerker with few redeeming qualities. M.J. Fox was the perfect voice for Stuart and the rest of the talent was wasted. Hugh Laurie can be amazingly funny, but is not given the chance in this movie. It´s sugar-coated sugar and would hardly appeal to anyone over 7 years of age. See Toy Story, Monsters Inc. or Shrek instead. 3/10']


In [29]:
#
# Make a smaller dataset so we can go fast, splitting into train and test.  
#

ntrain_per_class = 1000
ntest_per_class = 200

imdb_data_small = imdb_data[:ntrain_per_class] + imdb_data[-ntrain_per_class:]
nutterances = len(imdb_data_small)

print('number of utterances =', nutterances)
print('number of + utterances =', len([x for x in imdb_data_small if x[0] == 'pos']))
print('number of - utterances =', len([x for x in imdb_data_small if x[0] == 'neg']))
print('data[0] =', imdb_data_small[0])
print('data[-1] =', imdb_data_small[-1])

n1 = ntrain_per_class
n2 = ntrain_per_class + ntest_per_class
imdb_data_test = imdb_data[n1:n2] + imdb_data[-n2:-n1]
print('number of test utterances =', len(imdb_data_test))
print('number of + test utterances =', len([x for x in imdb_data_test if x[0] == 'pos']))
print('number of - test utterances =', len([x for x in imdb_data_test if x[0] == 'neg']))


number of utterances = 2000
number of + utterances = 1000
number of - utterances = 1000
data[0] = ['pos', 'For a movie that gets no respect there sure are a lot of memorable quotes listed for this gem. Imagine a movie where Joe Piscopo is actually funny! Maureen Stapleton is a scene stealer. The Moroni character is an absolute scream. Watch for Alan "The Skipper" Hale jr. as a police Sgt.']
data[-1] = ['neg', 'Not that I dislike childrens movies, but this was a tearjerker with few redeeming qualities. M.J. Fox was the perfect voice for Stuart and the rest of the talent was wasted. Hugh Laurie can be amazingly funny, but is not given the chance in this movie. It´s sugar-coated sugar and would hardly appeal to anyone over 7 years of age. See Toy Story, Monsters Inc. or Shrek instead. 3/10']
number of test utterances = 400
number of + test utterances = 200
number of - test utterances = 200


### Pre-processing

To make things faster, we will pre-process the data (tokenization, POS tagging, lemmatization) with spaCy's English pipeline. If you have some time to waste, you can try to do that explictly with NLTK's tokenizer, POS tagger and lemmatizer.


In [4]:
#
# We will process the text with spaCy for tokenization, POS tagging and lemmatization. We will process each text
# in the database and create a new representation of the database as a list where each entry is a dictionary with
# label, tokens, POS Tags, lemmas.
#
# For efficiency reasons, we call spaCy's processing in a special way (with pipe() that enables parallelization)
# and disable the elements of the pipeline we will not use (i.e, depency parsing and named entity recognition).
# This results in a list of processed texts from which we can read the tokens, POS tags and lemmas as we did
# for lecture 1.
#

database = imdb_data_small
nutterances = len(database)

raw_texts = [x[1] for x in database]
processed_texts = list(process.pipe(raw_texts, disable=['parser', 'ner']))
    
# initialize output structure
data = []


for i in range(nutterances):
    buf = {}
    buf['label'] = database[i][0]
    buf['token'] = [t.text.lower() for t in processed_texts[i]]
    buf['pos'] = [t.tag_ for t in processed_texts[i]]
    buf['lemma'] = [t.lemma_ for t in processed_texts[i]]    # already lowercased
    
    data.append(buf)

print(data[0])
print(data[-1])

{'label': 'pos', 'token': ['for', 'a', 'movie', 'that', 'gets', 'no', 'respect', 'there', 'sure', 'are', 'a', 'lot', 'of', 'memorable', 'quotes', 'listed', 'for', 'this', 'gem', '.', 'imagine', 'a', 'movie', 'where', 'joe', 'piscopo', 'is', 'actually', 'funny', '!', 'maureen', 'stapleton', 'is', 'a', 'scene', 'stealer', '.', 'the', 'moroni', 'character', 'is', 'an', 'absolute', 'scream', '.', 'watch', 'for', 'alan', '"', 'the', 'skipper', '"', 'hale', 'jr', '.', 'as', 'a', 'police', 'sgt', '.'], 'pos': ['IN', 'DT', 'NN', 'WDT', 'VBZ', 'DT', 'NN', 'RB', 'RB', 'VBP', 'DT', 'NN', 'IN', 'JJ', 'NNS', 'VBN', 'IN', 'DT', 'NN', '.', 'VB', 'DT', 'NN', 'WRB', 'NNP', 'NNP', 'VBZ', 'RB', 'JJ', '.', 'NNP', 'NNP', 'VBZ', 'DT', 'NN', 'NN', '.', 'DT', 'NNP', 'NN', 'VBZ', 'DT', 'JJ', 'NN', '.', 'VB', 'IN', 'NNP', '``', 'DT', 'NNP', "''", 'NNP', 'NNP', 'NNP', 'IN', 'DT', 'NN', 'NNP', '.'], 'lemma': ['for', 'a', 'movie', 'that', 'get', 'no', 'respect', 'there', 'sure', 'be', 'a', 'lot', 'of', 'memorable'

### Choose vocabulary

The _vocabulary_ is the list of terms that we will consider to represent the documents. We'll limit ourselves to simple terms (as opposed to complex terms such as 'can opener' or 'neural network') and simply select the most frequent terms in the corpus.

We provide the function get_token_counts() to help  doing that. The function takes the database as input and returns a list of tokens therein (no filtering in initial version) with the number of times they appear. Output
is a dictionary with token: count sorted in descending order w.r.t. the number of occurrences in the database 

Note that many toolkits for NLP provide a sort of equivalent function, e.g., 
   gensim.corpora.dictionary.Dictionary -- https://radimrehurek.com/gensim/corpora/dictionary.html#
   tf.keras.preprocessing.text.Tokenizer -- https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

But we'll do it with our own function to fully understand what's going on behind the scene.

In [5]:
def get_token_counts(idata, use_lemma = False, reserved = ()):
    '''
    Create vocabulary from a bunch of (tokenized) texts. If use_lemma == True, use lemma rather than
    tokens. The tuple 'reserved' can be provided to initialize the list of tokens with reserved
    names (e.g., [PAD], [UNK], [START], [END])
    
    No filtering on the tokens is implemented.
    
    Returns:
        - token to id mapping (dict)
        - token count (dict)
    '''

    tokcnt = {x: 0 for x in reserved} 
    
    kw = 'lemma' if use_lemma else 'token'
    
    for sample in idata:
        
        utterance = sample[kw]

        for i, token in enumerate(utterance):
            #
            # if we were to implement filters, e.g., on the POS tags, that's the place
            # where it could/should be done.
            #
            # e.g., if sample['pos'][i][0] not in ('N', 'V', 'J'): continue
            #
            tokcnt[token] = 1 if token not in tokcnt else tokcnt[token] + 1

    return dict(sorted(tokcnt.items(), key=lambda x: x[1], reverse = True))


count = get_token_counts(data)

#
# Pretty print a number of things
#
print('total number of tokens in dataset =', len(count))
print('most frequent tokens:')
for x in list(count.keys())[:20]:
    print(f"   {x:20}  {count[x]}")
print('\nleast frequent tokens:')
for x in list(count.keys())[-20:]:
    print(f"   {x:20}  {count[x]}")

#
# We select the top 2,000 tokens as our vocabulary to construct bag of words representations of the
# documents. The vocabulary will be dictionary mapping string to ids, the latter ranging from 0 to 1,999
#
nterms = 2000
vocab = {x: i for i,x in enumerate(list(count.keys())[:nterms])}

# ========================================================
# TODO
# 
# Comment the most and least frequent tokens and think about how you could get
# a list of tokens of interest other than by selecting the most frequent ones.
#

total number of tokens in dataset = 29331
most frequent tokens:
   the                   26264
   ,                     21180
   .                     19143
   a                     13139
   and                   13060
   of                    11648
   to                    11002
   is                    8892
   in                    7479
   it                    7427
   i                     6857
   this                  5930
   that                  5899
   "                     5135
   's                    4961
   -                     4438
   was                   4162
   /><br                 4104
   as                    3668
   for                   3512

least frequent tokens:
   castles               1
   too)                  1
   evelyn                1
   blanc                 1
   />zero/10             1
   cigars                1
   constitute            1
   afleck                1
   gigli.<br             1
   pointlessly           1
   parody.<br            1
   reple

### Compute idf weighting
 
Inverse document frequency (idf) is proportional to the number of samples in the database that contain a token. We want to compute that for every token in the vocabulary.

In [6]:
def get_num_doc(idata, token, use_lemma = False):
    '''
    Returns the number of samples in data that contains the specified token. If use_lemma is True,
    search in the lemma field, else in the token field.
    
    Note: if a filter is implemented in get_token_counts(), this function should be adapted accordingly
    because the wordform can become ambiguous.
    '''
    
    kw = 'lemma' if use_lemma else 'token'
    
    return len([x for x in idata if token in x[kw]])


#
# Make an idf dictionary where we store the idf for each token
#

idf = np.zeros(len(vocab), dtype=np.float64)
for term, idx in vocab.items():
    idf[idx] = np.log10(nutterances / get_num_doc(data, term))

#
# Pretty print for control
#
w = list(vocab.keys())[10]
print('token =', w, 'noccs =', get_num_doc(data, w), 'idf =', idf[10])

w = list(vocab.keys())[-10]
print('token =', w, 'noccs =', get_num_doc(data, w), 'idf =', idf[-10])


token = i noccs = 1592 idf = 0.09908693226233099
token = pretentious noccs = 22 idf = 1.9586073148417749


### Convert utterances to tf-idf vectors

We first provide a function get_num_occ() that returns the number of occurrences of a given token in an utterance. Here again, this is to be adapted if you're vocabulary construction implies filtering on POS.

We then use this function within a function tf_idf_vectorize() that converts a document into a tf-idf vector.

In [7]:

def get_num_occ(utterance, token, use_lemma = False):
    '''
    Return number of occurrences of a token in an utterance. 
    '''
    kw = 'lemma' if use_lemma else 'token'
    
    filtered = list(filter(lambda x: x == token, utterance[kw]))
    
    return len(filtered)

print(data[0]['token'])
for w in ('the', 'a', 'skipper'):
    idx = vocab[w] if w in vocab else -1
    print('  word {:10}: idx={}  noccs={}  idf={}'.format(w, idx, get_num_occ(data[0], w), idf[idx] if idx != -1 else ''))

# X = np.empty((nutterances, ntokens), dtype=np.float64)

['for', 'a', 'movie', 'that', 'gets', 'no', 'respect', 'there', 'sure', 'are', 'a', 'lot', 'of', 'memorable', 'quotes', 'listed', 'for', 'this', 'gem', '.', 'imagine', 'a', 'movie', 'where', 'joe', 'piscopo', 'is', 'actually', 'funny', '!', 'maureen', 'stapleton', 'is', 'a', 'scene', 'stealer', '.', 'the', 'moroni', 'character', 'is', 'an', 'absolute', 'scream', '.', 'watch', 'for', 'alan', '"', 'the', 'skipper', '"', 'hale', 'jr', '.', 'as', 'a', 'police', 'sgt', '.']
  word the       : idx=0  noccs=2  idf=0.003926345514724656
  word a         : idx=3  noccs=5  idf=0.015247721884586453
  word skipper   : idx=-1  noccs=1  idf=


In [13]:

def tf_idf_vectorize(utt, voc, idf, use_lemma = False):
    '''
    Convert utterance utt to tf-idf vector with vocabulary in voc.
    
    Note that this is by far not an efficient implementation but you can at least follow 
    the different steps.
    '''
    
    vec = np.zeros(len(voc), dtype=np.float64) 
    
    # get number of occurrences of each term -- remember the vocabulary is a mapping from string to index/dimension
    for term, idx in voc.items():
        vec[idx] = get_num_occ(utt, term, use_lemma)
    
    # normalize number of occurrences to get a term frequency
    vec = vec / sum(vec)
    
    # multiply tf by idf

    return np.multiply(vec, idf)

print(data[0]['token'])
print(tf_idf_vectorize(data[0], vocab, idf))


['for', 'a', 'movie', 'that', 'gets', 'no', 'respect', 'there', 'sure', 'are', 'a', 'lot', 'of', 'memorable', 'quotes', 'listed', 'for', 'this', 'gem', '.', 'imagine', 'a', 'movie', 'where', 'joe', 'piscopo', 'is', 'actually', 'funny', '!', 'maureen', 'stapleton', 'is', 'a', 'scene', 'stealer', '.', 'the', 'moroni', 'character', 'is', 'an', 'absolute', 'scream', '.', 'watch', 'for', 'alan', '"', 'the', 'skipper', '"', 'hale', 'jr', '.', 'as', 'a', 'police', 'sgt', '.']
[0.00016026 0.         0.00055744 ... 0.         0.         0.        ]


In [17]:
X = np.zeros((nutterances, nterms), dtype=np.float64)
for i in tqdm(range(nutterances)):
    X[i,:] = tf_idf_vectorize(data[i], vocab, idf)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:49<00:00, 40.01it/s]


### Train k-nn index to be able to retrieve k-nearest neighbors easily and test on test data

We use the training data in X to train a k-nn search with cosine distance, prepare test data and run classification.

In [42]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=5, metric='cosine').fit(X)

#
# What if we search for the k-nearest neighbors of each of the samples in the dataset?
#
dist, idx = knn.kneighbors(X)
print('For utterance 0, the closest points are at indices:', idx[0])
print('For utterance 0, the closest points are at distance:', dist[0])


For utterance 0, the closest points are at indices: [   0  884 1307 1429  232]
For utterance 0, the closest points are at distance: [0.         0.81670644 0.81773102 0.81989237 0.8217136 ]


In [30]:
#
# Let's prepare test data as we did for train data
#

# Step 1. tokenize, POS tag and lemmatize test data
database = imdb_data_test
ntests = len(database)

test_raw_texts = [x[1] for x in database]
test_processed_texts = list(process.pipe(test_raw_texts, disable=['parser', 'ner']))
test_data = []

for i in range(ntests):
    buf = {}
    buf['label'] = database[i][0]
    buf['token'] = [t.text.lower() for t in test_processed_texts[i]]
    buf['pos'] = [t.tag_ for t in test_processed_texts[i]]
    buf['lemma'] = [t.lemma_ for t in test_processed_texts[i]]    # already lowercased
    
    test_data.append(buf)

# Step 2. Vectorize test data
Y = np.zeros((ntests, nterms), dtype=np.float64)
for i in tqdm(range(ntests)):
    Y[i,:] = tf_idf_vectorize(test_data[i], vocab, idf)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:10<00:00, 38.92it/s]


In [44]:

def get_class(neighbors, th):
    '''
    Return class given the neighbors, assuming all neighbor indices below th are positive ones, 
    the remaining ones being negative.
    '''
    
    npos = len(list(filter(lambda i: i < th, neighbors))) # count the number of positive neighbors
    nneg = len(neighbors) - npos

    return 'pos' if npos > nneg else 'neg'
 
    
dist, idx = knn.kneighbors(Y)

print('For test utterance 0, the closest points are at indices:', idx[0])
print('For test utterance 0, the closest points are at distance:', dist[0])
print('Class for test utterance 0:', get_class(idx[0], ntrain_per_class))    


nok = [0, 0]
for i in tqdm(range(ntests)):
    c = get_class(idx[i], ntrain_per_class)
    if i < ntest_per_class and c == 'pos':
        nok[0] += 1
    elif i >= ntest_per_class and c == 'neg':
        nok[1] += 1

print('% + utterances correct = {:.2f}'.format(100 * nok[0] / ntest_per_class))
print('% - utterances correct = {:.2f}'.format(100 * nok[1] / ntest_per_class))
print('% utterances correct = {:.2f}'.format(100 * (nok[0] + nok[1]) / (2 * ntest_per_class)))


For test utterance 0, the closest points are at indices: [ 708  604 1431  275 1571]
For test utterance 0, the closest points are at distance: [0.74777752 0.76990411 0.78718233 0.79550038 0.81435515]
Class for test utterance 0: pos


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 400/400 [00:00<00:00, 155806.24it/s]

% + utterances correct = 70.50
% - utterances correct = 62.50
% utterances correct = 66.50


### TODO

We've been through the whole process: 
  1. utterance processing: tokenization, POS tagging, lemmmatization
  2. term selection: in this example, simply the most frequent tokens
  3. utterance vectorization: tf-idf weighting of the index term
  4. train k-nn index: train index structure for fast k-nn search
  5. k-nn classification: classify test data with k-nn classifier

Now it's your turn to improve things via better term selection. Up to you!